binary classification and multiclass classification are the results of the technique employed to solve the classification problem in this case.
its sigmoid and logistic funtion-

In [1]:
from sklearn.metrics import accuracy_score,auc,classification_report

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
import matplotlib.pyplot as plt

In [3]:
df_test=pd.read_csv('/home/amal/NoteBook/Dataset/loan_sanction_test.csv')
df_train=pd.read_csv('/home/amal/NoteBook/Dataset/loan_sanction_train.csv')
frame=[df_test,df_train]
data=pd.concat(frame)
data = data.drop(['Loan_ID'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             957 non-null    object 
 1   Married            978 non-null    object 
 2   Dependents         956 non-null    object 
 3   Education          981 non-null    object 
 4   Self_Employed      926 non-null    object 
 5   ApplicantIncome    981 non-null    int64  
 6   CoapplicantIncome  981 non-null    float64
 7   LoanAmount         954 non-null    float64
 8   Loan_Amount_Term   961 non-null    float64
 9   Credit_History     902 non-null    float64
 10  Property_Area      981 non-null    object 
 11  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(7)
memory usage: 99.6+ KB


In [4]:
import numpy as np

# calculate the percentage of missing values for each column
missing_values_percent = data.isnull().sum() / len(data) * 100
print(missing_values_percent)

# create a list of columns to drop
columns_to_drop = missing_values_percent[missing_values_percent > 50].index.tolist()

# drop the columns with more than 50% missing values
data = data.drop(columns_to_drop, axis=1)

# fill the remaining missing values
for column in data.columns:
    if data[column].dtype == 'object':
        # fill missing values with the mode of the column
        data[column] = data[column].fillna(data[column].mode()[0])
    elif data[column].dtype in ['int64', 'float64']:
        # fill missing values with the mean of the column
        data[column] = data[column].fillna(data[column].mean())


Gender                2.446483
Married               0.305810
Dependents            2.548420
Education             0.000000
Self_Employed         5.606524
ApplicantIncome       0.000000
CoapplicantIncome     0.000000
LoanAmount            2.752294
Loan_Amount_Term      2.038736
Credit_History        8.053007
Property_Area         0.000000
Loan_Status          37.410805
dtype: float64


In [5]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
encoder = LabelEncoder()

# Encode the "Loan_Status" column
data['Loan_Status'] = encoder.fit_transform(data['Loan_Status'])
y=data['Loan_Status']
# Check the unique values of the "Loan_Status" column after encoding
# print(data['Loan_Status'].unique())
# Output: [1 0]
data['Loan_Status'].unique()


array([1, 0])

In [6]:
data.Loan_Status.unique()

array([1, 0])

In [7]:
categorical_columns = []

# iterate through each column in the dataset
for column in data.columns:
    # check if the column is of type 'object' (usually a categorical column)
    if data[column].dtype == 'object':
        # add the column to the list
        categorical_columns.append(column)

# create a new dataset with only the categorical columns
categorical_df = pd.DataFrame(data[categorical_columns])
categorical_df

,Gender,Married,Dependents,Education,Self_Employed,Property_Area
0,Male,Yes,0,Graduate,No,Urban
1,Male,Yes,1,Graduate,No,Urban
2,Male,Yes,2,Graduate,No,Urban
3,Male,Yes,2,Graduate,No,Urban
4,Male,No,0,Not Graduate,No,Urban
...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,Rural
610,Male,Yes,3+,Graduate,No,Rural
611,Male,Yes,1,Graduate,No,Urban
612,Male,Yes,2,Graduate,No,Urban


In [8]:
import pandas as pd

# One-hot encode the categorical columns
encoded_categorical_df = pd.get_dummies(categorical_df, columns=['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area'])


In [9]:
encoded_categorical_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 613
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Gender_Female            981 non-null    uint8
 1   Gender_Male              981 non-null    uint8
 2   Married_No               981 non-null    uint8
 3   Married_Yes              981 non-null    uint8
 4   Dependents_0             981 non-null    uint8
 5   Dependents_1             981 non-null    uint8
 6   Dependents_2             981 non-null    uint8
 7   Dependents_3+            981 non-null    uint8
 8   Education_Graduate       981 non-null    uint8
 9   Education_Not Graduate   981 non-null    uint8
 10  Self_Employed_No         981 non-null    uint8
 11  Self_Employed_Yes        981 non-null    uint8
 12  Property_Area_Rural      981 non-null    uint8
 13  Property_Area_Semiurban  981 non-null    uint8
 14  Property_Area_Urban      981 non-null    uint8
dtypes: uin

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             981 non-null    object 
 1   Married            981 non-null    object 
 2   Dependents         981 non-null    object 
 3   Education          981 non-null    object 
 4   Self_Employed      981 non-null    object 
 5   ApplicantIncome    981 non-null    int64  
 6   CoapplicantIncome  981 non-null    float64
 7   LoanAmount         981 non-null    float64
 8   Loan_Amount_Term   981 non-null    float64
 9   Credit_History     981 non-null    float64
 10  Property_Area      981 non-null    object 
 11  Loan_Status        981 non-null    int64  
dtypes: float64(4), int64(2), object(6)
memory usage: 99.6+ KB


In [11]:
numeri_col=data.loc[:,['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History']]


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 613
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             981 non-null    object 
 1   Married            981 non-null    object 
 2   Dependents         981 non-null    object 
 3   Education          981 non-null    object 
 4   Self_Employed      981 non-null    object 
 5   ApplicantIncome    981 non-null    int64  
 6   CoapplicantIncome  981 non-null    float64
 7   LoanAmount         981 non-null    float64
 8   Loan_Amount_Term   981 non-null    float64
 9   Credit_History     981 non-null    float64
 10  Property_Area      981 non-null    object 
 11  Loan_Status        981 non-null    int64  
dtypes: float64(4), int64(2), object(6)
memory usage: 99.6+ KB


In [13]:
# numeric_col.loc[:, ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Credit_History']] = scaler.fit_transform(numeric_col[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Credit_History']])


In [14]:
# numeric_col.info()

In [15]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numeri_col[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Credit_History']] = scaler.fit_transform(numeri_col[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Credit_History']])
# Dataset=scaler.fit_transform(concatenated_df)

In [16]:
concatenated_df = pd.concat([encoded_categorical_df, numeri_col], axis=1)


In [17]:
concatenated_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 981 entries, 0 to 613
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender_Female            981 non-null    uint8  
 1   Gender_Male              981 non-null    uint8  
 2   Married_No               981 non-null    uint8  
 3   Married_Yes              981 non-null    uint8  
 4   Dependents_0             981 non-null    uint8  
 5   Dependents_1             981 non-null    uint8  
 6   Dependents_2             981 non-null    uint8  
 7   Dependents_3+            981 non-null    uint8  
 8   Education_Graduate       981 non-null    uint8  
 9   Education_Not Graduate   981 non-null    uint8  
 10  Self_Employed_No         981 non-null    uint8  
 11  Self_Employed_Yes        981 non-null    uint8  
 12  Property_Area_Rural      981 non-null    uint8  
 13  Property_Area_Semiurban  981 non-null    uint8  
 14  Property_Area_Urban      9


# MODEL CREATION

In [18]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(concatenated_df, data['Loan_Status'], test_size=0.25, random_state=42)


In [19]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()

In [20]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier()

In [26]:
clf.fit(X_train,y_train)

RandomForestClassifier()

In [27]:
pred=clf.predict(X_test)
print(accuracy_score(y_test,pred))

0.8130081300813008


In [22]:
lr.fit(X_train,y_train)

/home/amal/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [23]:
prediction=lr.predict(X_test)

In [24]:
prediction

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1])

In [25]:
from sklearn.metrics import classification_report,confusion_matrix

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))


NameError: name 'y_pred' is not defined

In [ ]:
# pip install mlxtend

In [ ]:
from mlxtend.plotting import plot_decision_regions

In [ ]:
# plot_decision_regions(X_test, y_test, clf=lr)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(concatenated_df, data['Loan_Status'], test_size=0.2, random_state=42)

# Define the models to be tested
models = {'Logistic Regression': LogisticRegression(),
          'KNN': KNeighborsClassifier(),
          'Decision Tree': DecisionTreeClassifier(),
          'SVM': SVC(),
          'Random Forest': RandomForestClassifier()}

# Iterate through the models and print their performance
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f'{model_name} Performance:')
    print(f'Accuracy: {accuracy:.2f}')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')
    print()


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for tuning
param_grid = {'C': [0.1, 1, 10], 
              'penalty': ['l1', 'l2'], 
              'fit_intercept': [True, False], 
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag']}

# Instantiate the logistic regression model
log_reg = LogisticRegression()

# Perform grid search
grid_search = GridSearchCV(log_reg, param_grid, cv=10, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)
